In [1]:
import keras as k
import numpy
import os

Using TensorFlow backend.


In [2]:
with open("example-keras-model-files/KERAS_conv1d.json") as jsonFile:
    jsonModel = jsonFile.read()
model = k.models.model_from_json(jsonModel)
model.load_weights("example-keras-model-files/KERAS_conv1d_weights.h5")
model.summary()

ValueError: Unknown constraint: ZeroSomeWeights